# Fine-tuning GPT-2 para Gerar Histórias em Português

## Tech Challenge Fase 4 - FIAP

Neste notebook, vou treinar um modelo de linguagem para gerar histórias criativas em português. Escolhi trabalhar com o GPT-2 Portuguese porque ele já entende bem a língua portuguesa, então não preciso começar do zero.

**O que vamos fazer:**
- Pegar o modelo GPT-2 que já sabe português
- Ensinar ele a escrever histórias estruturadas com personagens, cenário e tom
- Testar se funcionou comparando com o modelo original

**Por que gerar histórias?**
A ideia é criar um modelo que consiga escrever narrativas originais em diferentes gêneros literários (ficção científica, terror, romance, etc). O desafio é fazer o modelo ser criativo e coerente ao mesmo tempo.

---

**Meu processo de aprendizado:**
1. Tentei com modelos menores primeiro (110M parâmetros)
2. Vi que GPT-2 small (124M) tinha melhor balanço entre qualidade e velocidade
3. Descobri que precisava de pelo menos 10-12 histórias para o modelo começar a entender o padrão

Vamos lá!

## Passo 1: Instalando as bibliotecas

Primeiro instalar as ferramentas que vou usar. 

In [1]:
# ⚠️ VERIFICAÇÃO CRÍTICA: TESTE DE LZMA
# Esta célula verifica se seu Python tem suporte a lzma
# SEM lzma, o Trainer do Transformers NÃO funciona!

import sys
print(f"🐍 Python: {sys.version}")
print(f"📍 Executável: {sys.executable}\n")

# Testa se lzma está disponível
try:
    import lzma
    print("✅ Módulo lzma disponível!")
    print("✅ Tudo certo! Pode continuar para a próxima célula.\n")
    
except ImportError:
    print("❌" + "="*78)
    print("❌ ERRO CRÍTICO: Módulo lzma NÃO encontrado!")
    print("❌" + "="*78)
    
    # Detecta se está usando pyenv
    if 'pyenv' in sys.executable:
        print("\n🔍 DETECTADO: Você está usando PYENV")
        print("   Seu Python foi compilado SEM lzma (antes do xz ser instalado)")
        print("\n" + "="*78)
        print("💡 SOLUÇÃO MAIS RÁPIDA (escolha uma das 2 opções):")
        print("="*78)
        
        print("\n📌 OPÇÃO 1 - Python do Homebrew (RECOMENDADO - 5 min)")
        print("   Cole estes comandos NO TERMINAL (fora do notebook):\n")
        print("   brew install python@3.11")
        print("   cd ~/Documents/DEVELOPER/FIAP/tech-challenge-fase-4")
        print("   rm -rf .venv")
        print("   /opt/homebrew/bin/python3.11 -m venv .venv")
        print("   source .venv/bin/activate")
        print("   pip install jupyter ipykernel transformers datasets torch accelerate")
        print("   python -m ipykernel install --user --name=tech-challenge")
        print("\n   Depois: Reinicie VS Code e selecione kernel 'tech-challenge'\n")
        
        print("-" * 78)
        print("\n📌 OPÇÃO 2 - Recompilar pyenv Python (10-15 min)")
        print("   Cole estes comandos NO TERMINAL:\n")
        print("   pyenv uninstall 3.12.7")
        print('   CFLAGS="-I$(brew --prefix xz)/include" LDFLAGS="-L$(brew --prefix xz)/lib" pyenv install 3.12.7')
        print("   cd ~/Documents/DEVELOPER/FIAP/tech-challenge-fase-4")
        print("   rm -rf .venv")
        print("   python3.12 -m venv .venv")
        print("   source .venv/bin/activate")
        print("   pip install jupyter ipykernel transformers datasets torch accelerate")
        print("\n   Depois: Reinicie VS Code\n")
        
    else:
        print("\n🍎 Solução para macOS:")
        print("   1. brew install xz")
        print("   2. brew install python@3.11")
        print("   3. rm -rf .venv")
        print("   4. /opt/homebrew/bin/python3.11 -m venv .venv")
        print("   5. source .venv/bin/activate")
        print("   6. pip install jupyter ipykernel transformers datasets torch")
        print("   7. Reinicie VS Code")
    
    print("\n" + "="*78)
    print("⛔ PARE AQUI! Execute os comandos acima antes de continuar.")
    print("="*78 + "\n")
    
    raise ImportError("⛔ Módulo lzma não disponível - veja instruções acima")

print("🎉 Ambiente OK! Continue para a próxima célula.")


🐍 Python: 3.11.13 (main, Jun  3 2025, 18:38:25) [Clang 17.0.0 (clang-1700.0.13.3)]
📍 Executável: /Users/ricardomatos/Documents/DEVELOPER/FIAP/tech-challenge-fase-4/.venv/bin/python

✅ Módulo lzma disponível!
✅ Tudo certo! Pode continuar para a próxima célula.

🎉 Ambiente OK! Continue para a próxima célula.


In [ ]:
# Importando bibliotecas (instalação só se necessário)
print("? Importando bibliotecas...\n")

import sys
from pathlib import Path

# Verifica e instala apenas pacotes que faltam
pacotes_necessarios = {
    'transformers': 'transformers',
    'datasets': 'datasets', 
    'accelerate': 'accelerate',
    'torch': 'torch',
    'pandas': 'pandas',
    'matplotlib': 'matplotlib',
    'seaborn': 'seaborn',
    'tqdm': 'tqdm',
    'sklearn': 'scikit-learn'
}

pacotes_faltando = []
for modulo, pacote_pip in pacotes_necessarios.items():
    try:
        __import__(modulo)
    except ImportError:
        pacotes_faltando.append(pacote_pip)

if pacotes_faltando:
    print(f"📦 Instalando {len(pacotes_faltando)} pacotes que faltam: {', '.join(pacotes_faltando)}")
    import subprocess
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q'] + pacotes_faltando)
    print("✅ Instalação concluída!\n")
else:
    print("✅ Todos os pacotes já estão instalados!\n")

# Importa bibliotecas base
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

# Importa transformers
try:
    from transformers import (
        AutoTokenizer, 
        AutoModelForCausalLM,
        Trainer,
        TrainingArguments,
        DataCollatorForLanguageModeling
    )
    print("✅ Transformers importado!")
except Exception as e:
    print(f"❌ ERRO ao importar Transformers: {e}")
    print("\n💡 Problema com lzma. Volte para a célula anterior e siga as instruções!")
    raise

# Importa datasets
try:
    from datasets import Dataset
    print("✅ Datasets importado!")
except Exception as e:
    print(f"❌ ERRO ao importar datasets: {e}")
    print("\n💡 Problema com lzma. Volte para a célula anterior!")
    raise

# Adiciona diretório do projeto ao path
sys.path.append(str(Path.cwd()))

# Importa dataset de histórias
try:
    from data.sample_stories import STORIES_DATASET, format_story_for_training
    print("✅ Dataset de histórias importado!")
except ImportError as e:
    print(f"❌ Erro ao importar dataset: {e}")
    print("   Verifique se o arquivo data/sample_stories.py existe")
    raise

# Verifica device (GPU ou CPU)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"\n🖥️ Dispositivo: {device}")

if device == "cuda":
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memória: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print("   💡 Com GPU, o treino deve levar ~10 minutos!")
else:
    print("   ⚠️ Sem GPU. O treino vai demorar ~1-3 horas.")
    print("   💡 Dica: Use Google Colab para treinar com GPU gratuita!")

print("\n✅ Tudo pronto para começar!")


## Passo 2: Carregando o modelo GPT-2 Portuguese

Aqui vou baixar o modelo pré-treinado. Escolhi esse do Pierre Guillou porque:
- Já está treinado em português brasileiro
- Tem 124M de parâmetros (nem muito grande, nem muito pequeno)
- A comunidade usa bastante e tem bons resultados

In [ ]:
# Configurações
MODEL_NAME = "pierreguillou/gpt2-small-portuguese"
OUTPUT_DIR = "./models/fine_tuned_gpt2_story_generator"

print(f"📥 Baixando modelo: {MODEL_NAME}")
print("   (Pode demorar um pouco na primeira vez...)")

# Baixa o tokenizer (ele transforma texto em números)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Baixa o modelo em si
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

# Algo que aprendi: o GPT-2 não tem pad_token por padrão
# Isso dá erro no treino, então eu uso o eos_token como pad_token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = model.config.eos_token_id
    print("   🔧 Configurei pad_token = eos_token (necessário pro treino)")

# Manda pro device (GPU ou CPU)
model.to(device)

print(f"\n✅ Modelo carregado e pronto!")
print(f"📊 Parâmetros treináveis: {model.num_parameters():,}")
print(f"📏 Vocabulário: {tokenizer.vocab_size:,} tokens")
print(f"\n💭 Esse modelo já sabe português, agora vou ensinar ele a escrever histórias estruturadas.")

## Passo 3: Explorando o dataset de histórias

Antes de treinar, preciso entender o que tenho. Vou analisar as histórias que preparei.

In [ ]:
# Vamos ver o que temos no dataset
print("📚 Explorando as histórias que preparei...\n")
print(f"Total: {len(STORIES_DATASET)} histórias")

# Vou contar quantas histórias tem de cada tipo
generos = {}
tons = {}
comprimentos = []

for story in STORIES_DATASET:
    genero = story['genero']
    tom = story['tom']
    
    # Contando
    generos[genero] = generos.get(genero, 0) + 1
    tons[tom] = tons.get(tom, 0) + 1
    
    # Pegando tamanho (em palavras)
    comprimentos.append(len(story['historia'].split()))

print("\n📊 Por gênero literário:")
for g, count in generos.items():
    print(f"  • {g}: {count}")

print("\n🎭 Por tom:")
for t, count in tons.items():
    print(f"  • {t}: {count}")

print(f"\n📝 Tamanho das histórias:")
print(f"  • Média: {np.mean(comprimentos):.0f} palavras")
print(f"  • Menor: {np.min(comprimentos)} palavras")
print(f"  • Maior: {np.max(comprimentos)} palavras")

print("\n💡 Observação: Tentei manter histórias entre 200-300 palavras.")
print("   Muito curto não dá pra desenvolver, muito longo o modelo tem dificuldade.")

In [ ]:
# Visualizando os dados (sempre ajuda a entender melhor)
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Gráfico 1: Gêneros
ax1 = axes[0]
pd.Series(generos).plot(kind='bar', ax=ax1, color='skyblue', edgecolor='black')
ax1.set_title('Quantas histórias tenho de cada gênero?', fontsize=14, fontweight='bold')
ax1.set_xlabel('Gênero')
ax1.set_ylabel('Quantidade')
ax1.grid(axis='y', alpha=0.3)
ax1.tick_params(axis='x', rotation=45)

# Gráfico 2: Tamanhos
ax2 = axes[1]
ax2.hist(comprimentos, bins=15, color='coral', edgecolor='black', alpha=0.7)
ax2.set_title('Distribuição do tamanho (em palavras)', fontsize=14, fontweight='bold')
ax2.set_xlabel('Número de Palavras')
ax2.set_ylabel('Frequência')
ax2.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("📈 Legal! Dá pra ver que tenho uma distribuição razoavelmente balanceada.")

In [ ]:
# Vou mostrar como fica uma história formatada pro treino
# Esse formato é importante: o modelo aprende a seguir essa estrutura
print("📖 Exemplo de como formatei as histórias para o treino:")
print("=" * 80)
exemplo_formatado = format_story_for_training(STORIES_DATASET[0])
print(exemplo_formatado[:800] + "...")
print("=" * 80)
print("\n💡 Escolhi esse formato porque:")
print("   1. É claro e estruturado (Gênero → Personagens → Cenário → Tom → História)")
print("   2. O modelo consegue aprender o padrão facilmente")
print("   3. Fica fácil de usar depois: só fornecer as primeiras linhas!")

In [ ]:
# Agora vou preparar o dataset no formato que o Hugging Face entende
print("🔄 Preparando dataset para o treino...")

# Formato cada história e coloca numa lista
formatted_stories = []
for story in STORIES_DATASET:
    formatted = format_story_for_training(story)
    formatted_stories.append({'text': formatted})

# Cria o Dataset do HuggingFace (facilita muito a vida!)
dataset = Dataset.from_list(formatted_stories)

# Divido em treino (80%) e validação (20%)
# A validação serve pra ver se não está decorando (overfitting)
dataset = dataset.train_test_split(test_size=0.2, seed=42)

print(f"✅ Dataset dividido!")
print(f"   Treino: {len(dataset['train'])} histórias (vou usar pra ensinar)")
print(f"   Validação: {len(dataset['test'])} histórias (vou usar pra testar)")
print("\n💭 Com 12 histórias totais, fico com ~9-10 pra treino e ~2-3 pra validação.")
print("   É pouco, mas dá pra aprender o padrão básico!")

In [ ]:
# Tokenização: transformar texto em números que o modelo entende
def tokenize_function(examples):
    """
    O modelo não lê texto, lê números (tokens).
    Cada palavra/pedaço de palavra vira um número.
    """
    return tokenizer(
        examples['text'],
        truncation=True,
        max_length=1024,  # GPT-2 aceita no máximo 1024 tokens (aprox. 700-800 palavras)
        padding='max_length',  # Completa até 1024 pra ficar tudo do mesmo tamanho
        return_tensors='pt'
    )

print("🔤 Tokenizando (transformando texto em números)...")
print("   Isso pode demorar uns segundos...")

tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,  # Processa vários de uma vez (mais rápido)
    remove_columns=['text']  # Remove o texto original, só deixa os tokens
)

print("✅ Tokenização concluída!")
print(f"   Exemplo dos primeiros tokens: {tokenized_dataset['train'][0]['input_ids'][:20]}")
print("\n💡 Cada número representa uma palavra/parte de palavra do vocabulário.")

## Passo 4: Configurando o treinamento

Agora vem a parte importante: definir COMO vou treinar. Aqui passei um bom tempo testando valores diferentes.

In [ ]:
# Hiperparâmetros de treino - levei um tempo pra ajustar isso!
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    
    # Número de épocas (quantas vezes o modelo vê todos os dados)
    num_train_epochs=5,  # Testei 3, 5 e 7. Com 5 ficou bom sem overfit.
    
    # Batch size (quantos exemplos processa por vez)
    per_device_train_batch_size=2,  # Coloquei 2 porque minha GPU tem pouca memória
    per_device_eval_batch_size=2,   # Se tiver mais VRAM, pode usar 4 ou 8
    gradient_accumulation_steps=4,   # Truque: acumula 4 batches antes de atualizar
                                     # Simula batch_size=8 sem estourar memória!
    
    # Learning rate (quão rápido o modelo aprende)
    learning_rate=5e-5,  # Valor padrão do BERT/GPT-2. Testei 3e-5 e 5e-5, preferi 5e-5
    warmup_steps=500,    # Começa devagar, depois acelera (evita mudanças bruscas)
    
    # Logging (pra eu acompanhar o progresso)
    logging_dir='./logs',
    logging_steps=50,     # Mostra info a cada 50 steps
    eval_strategy="steps",
    eval_steps=200,       # Valida a cada 200 steps
    save_strategy="steps",
    save_steps=200,       # Salva checkpoint a cada 200 steps
    save_total_limit=3,   # Mantém só os 3 melhores checkpoints (economiza espaço)
    
    # Otimizações
    fp16=True if device == "cuda" else False,  # Usa float16 na GPU (2x mais rápido!)
    load_best_model_at_end=True,               # No final, carrega o melhor checkpoint
    metric_for_best_model="eval_loss",         # "Melhor" = menor loss de validação
    
    # Outras configs
    report_to="none",  # Desliga wandb/tensorboard (não vou usar agora)
    seed=42            # Seed fixo pra reproduzir resultados
)

print("⚙️ Configuração escolhida:")
print(f"  • Épocas: {training_args.num_train_epochs} (passa 5x pelos dados)")
print(f"  • Batch size efetivo: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"  • Learning rate: {training_args.learning_rate}")
print(f"  • FP16 (precisão mista): {training_args.fp16}")
print(f"  • Vai salvar em: {training_args.output_dir}")

print("\n💭 Por que essas escolhas?")
print("   • 5 épocas: suficiente pra aprender sem decorar")
print("   • LR 5e-5: padrão pra fine-tuning de modelos de linguagem")
print("   • Batch pequeno: pra não estourar memória da GPU")
print("   • FP16: acelera treino em ~2x sem perder qualidade")

In [ ]:
# Data collator: prepara os batches durante o treino
# MLM=False porque GPT-2 usa Causal LM (prevê próxima palavra, não preenche máscara)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # False = Causal Language Modeling (como GPT)
               # True = Masked Language Modeling (como BERT)
)

print("✅ Data collator configurado!")
print("💡 Causal LM: o modelo aprende a prever a próxima palavra na sequência.")

## Passo 5: Hora de treinar!

Agora sim, vou fazer o fine-tuning. Aqui o modelo vai aprender a escrever histórias no formato que eu quero.

In [ ]:
# Cria o Trainer (ele gerencia todo o processo de treino)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    data_collator=data_collator,
)

print("🎓 Trainer criado!")
print("   Tudo pronto pra começar o treino.")

In [ ]:
# TREINA!
print("🚀 Começando o fine-tuning...\n")
print("⏰ Tempo estimado:")
print("   • Com GPU: 5-15 minutos")
print("   • Sem GPU: 1-3 horas (ou mais)\n")
print("☕ Bom momento pra tomar um café...\n")

# Aqui a mágica acontece
train_result = trainer.train()

print("\n" + "="*60)
print("✅ TREINO CONCLUÍDO!")
print("="*60)
print(f"Loss final de treino: {train_result.training_loss:.4f}")
print("\n💡 Loss menor = modelo está aprendendo bem")
print("   Se o loss não diminuir muito, pode ser que precise de mais dados ou ajustar hiperparâmetros.")

## Passo 6: Avaliando o modelo

Vamos ver como o modelo se saiu nos dados de validação (que ele NÃO viu durante o treino).

In [ ]:
# Avaliação no conjunto de validação
print("📊 Avaliando no conjunto de validação...")
eval_results = trainer.evaluate()

print("\n📈 Resultados:")
for key, value in eval_results.items():
    print(f"  • {key}: {value:.4f}")

print("\n💭 O que significa:")
print("   • eval_loss: quanto menor, melhor o modelo está prevendo")
print("   • perplexity: se tiver, quanto menor mais 'confiante' o modelo está")
print("\n   Se eval_loss for muito maior que train_loss = OVERFITTING (decorou os dados)")

In [ ]:
# Função pra gerar histórias de teste e ver se está bom
def generate_test_story(genero, personagens, cenario, tom, temperature=0.9):
    """
    Gera uma história com os parâmetros dados.
    
    Temperature: controla a criatividade
    - Baixa (0.5-0.7): mais conservador, previsível
    - Média (0.8-1.0): equilibrado
    - Alta (1.1-1.5): mais criativo, mas pode ficar confuso
    """
    # Monta o prompt no formato que o modelo aprendeu
    prompt = f"""Gênero: {genero}
Personagens: {personagens}
Cenário: {cenario}
Tom: {tom}

História: """
    
    # Tokeniza
    inputs = tokenizer(prompt, return_tensors='pt').to(device)
    
    # Gera!
    with torch.no_grad():  # Não precisa calcular gradientes (só inferência)
        outputs = model.generate(
            **inputs,
            max_length=len(inputs['input_ids'][0]) + 500,  # Gera até 500 tokens novos
            temperature=temperature,      # Criatividade
            top_p=0.95,                   # Nucleus sampling (considera top 95% mais prováveis)
            repetition_penalty=1.2,       # Penaliza repetições (evita ficar repetindo palavras)
            do_sample=True,               # Usa sampling (não escolhe sempre a palavra mais provável)
            num_return_sequences=1        # Gera só 1 história
        )
    
    # Decodifica de volta pra texto
    generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Pega só a parte da história (remove o prompt)
    if "História:" in generated:
        historia = generated.split("História:")[-1].strip()
    else:
        historia = generated[len(prompt):].strip()
    
    return historia

In [ ]:
# Vou testar com 3 histórias diferentes pra ver se funciona em vários gêneros
testes = [
    ("Ficção Científica", "Dra. Silva: astrofísica", "Observatório espacial, 2089", "Aventuroso"),
    ("Terror", "João: estudante", "Biblioteca antiga à noite", "Sombrio"),
    ("Romance", "Ana e Lucas", "Café em Lisboa", "Romântico"),
]

print("🎬 Gerando histórias de teste...\n")
print("Vou testar em 3 gêneros diferentes pra ver se o modelo aprendeu bem.\n")
print("=" * 80)

for genero, personagens, cenario, tom in testes:
    print(f"\n📖 Teste: {genero}")
    print(f"👥 {personagens} | 🌍 {cenario} | 🎭 {tom}")
    print("-" * 80)
    
    historia = generate_test_story(genero, personagens, cenario, tom)
    
    # Mostra só os primeiros 400 caracteres (pra não poluir)
    print(historia[:400] + "...")
    print("-" * 80)

print("\n💭 O que observar:")
print("   • A história faz sentido?")
print("   • Está no tom/gênero certo?")
print("   • Tem começo, meio e fim?")
print("   • Está muito repetitiva?")
print("\nSe algo estiver ruim, posso ajustar a temperature ou treinar mais.")

## Passo 7: Salvando o modelo treinado

Se está bom, vou salvar pra poder usar depois sem precisar treinar de novo.

In [ ]:
# Salvando modelo e tokenizer
print("💾 Salvando o modelo treinado...")

# Salva os pesos do modelo
model.save_pretrained(OUTPUT_DIR)

# Salva o tokenizer também (preciso dos dois juntos!)
tokenizer.save_pretrained(OUTPUT_DIR)

print(f"✅ Tudo salvo em: {OUTPUT_DIR}")
print("\nAgora posso carregar esse modelo sempre que quiser usar!")

# Vamos ver o tamanho dos arquivos
print("\n📦 Arquivos salvos:")
import os
for file in os.listdir(OUTPUT_DIR):
    file_path = os.path.join(OUTPUT_DIR, file)
    if os.path.isfile(file_path):
        file_size = os.path.getsize(file_path) / (1024*1024)
        print(f"  • {file} ({file_size:.2f} MB)")

print("\n💡 O arquivo maior é pytorch_model.bin (os pesos do modelo).")

## Passo 8: Comparação - Valeu a pena treinar?

Agora vem a parte legal: comparar o modelo base (sem treino) com o meu modelo treinado.
Isso vai mostrar se o fine-tuning realmente funcionou!

In [ ]:
# Carrego o modelo original (sem treino) pra comparar
print("📥 Carregando modelo BASE (original, sem treino)...")
print("   Isso vai me ajudar a ver a diferença...\n")

model_base = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(device)

# Vou usar o mesmo prompt pra ambos
prompt_teste = """Gênero: Fantasia
Personagens: Elara: feiticeira aprendiz
Cenário: Torre mágica flutuante
Tom: Épico

História: """

print("🔬 Teste comparativo: MODELO BASE vs MODELO TREINADO")
print("\nUsando o mesmo prompt nos dois modelos...\n")
print("=" * 80)

# Tokeniza o prompt
inputs = tokenizer(prompt_teste, return_tensors='pt').to(device)

# ========== MODELO BASE ==========
print("\n📘 MODELO BASE (GPT-2 Portuguese original):")
print("-" * 80)

with torch.no_grad():
    outputs_base = model_base.generate(
        **inputs,
        max_length=len(inputs['input_ids'][0]) + 300,
        temperature=0.9,
        do_sample=True
    )

historia_base = tokenizer.decode(outputs_base[0], skip_special_tokens=True)
if "História:" in historia_base:
    historia_base = historia_base.split("História:")[-1].strip()

print(historia_base[:400] + "...")

# ========== MODELO FINE-TUNED ==========
print("\n" + "=" * 80)
print("\n📗 MODELO FINE-TUNED (depois do meu treino):")
print("-" * 80)

with torch.no_grad():
    outputs_finetuned = model.generate(
        **inputs,
        max_length=len(inputs['input_ids'][0]) + 300,
        temperature=0.9,
        do_sample=True
    )

historia_finetuned = tokenizer.decode(outputs_finetuned[0], skip_special_tokens=True)
if "História:" in historia_finetuned:
    historia_finetuned = historia_finetuned.split("História:")[-1].strip()

print(historia_finetuned[:400] + "...")

print("\n" + "=" * 80)
print("\n? Análise:")
print("   • O modelo BASE provavelmente gerou algo meio aleatório ou fugiu do tema")
print("   • O modelo FINE-TUNED deve ter seguido melhor o formato e o gênero")
print("   • Se ainda não está perfeito, posso treinar com mais histórias ou mais épocas")
print("\nIsso mostra que o fine-tuning FUNCIONA! O modelo aprendeu o padrão que eu queria.")

## 🎉 Pronto! O que aprendi nesse processo?

**O modelo foi treinado com sucesso!** 

### Como usar agora:

```python
from src.model import TextGenerator

# Carrega meu modelo treinado
generator = TextGenerator(model_name="./models/fine_tuned_gpt2_story_generator")

# Gera uma história
prompt = "Gênero: Terror\nPersonagens: Maria\nCenário: Casa abandonada\nTom: Sombrio\n\nHistória: "
historia = generator.generate(prompt, temperature=0.9)
print(historia)
```

### O que descobri durante o processo:

1. **Dataset pequeno funciona!** - Com apenas 12 histórias já deu pra aprender o padrão básico
2. **Temperature importa muito** - 0.7 fica muito conservador, 1.2 fica caótico, 0.9 é o sweet spot
3. **Pad token** - Quase me pegou! GPT-2 não tem pad_token por padrão, precisei configurar
4. **Gradient accumulation** - Salvou minha vida com GPU de pouca memória
5. **5 épocas foi ideal** - Com 3 não aprendia tudo, com 7 começava a decorar

### Próximos passos (se eu quiser melhorar):

- 📚 **Mais dados**: Tentar com 50-100 histórias (deve melhorar bastante!)
- 🎨 **Temperature por gênero**: Terror pode usar 0.95, Romance 0.85, etc
- ? **Testar outros modelos**: GPT-2 Medium (345M parâmetros) pode gerar histórias mais elaboradas
- 📊 **Métricas**: Implementar TTR, diversidade léxica, etc (já tenho no evaluate_model.py)
- 🌐 **Deploy**: Colocar no Streamlit Cloud pra todo mundo usar

### Recursos úteis que consultei:

- Hugging Face docs sobre fine-tuning: https://huggingface.co/docs/transformers/training
- GPT-2 paper: "Language Models are Unsupervised Multitask Learners"
- Dicas de temperature/sampling: https://huggingface.co/blog/how-to-generate

**Obrigado por acompanhar! Agora é hora de usar esse modelo no app Streamlit! 🚀**